In [10]:
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install 'cupy-cuda80==4.0.0b4' 'chainer==4.0.0b4'
!pip install hyperopt==0.1
!pip install networkx==1.11

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcusparse8.0 is already the newest version (8.0.61-1).
libnvrtc8.0 is already the newest version (8.0.61-1).
libnvtoolsext1 is already the newest version (8.0.61-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [0]:
# !pip install hyperopt==0.1
# !pip install networkx==1.11

In [0]:
# !pip install --upgrade git+git://github.com/hyperopt/hyperopt.git

In [0]:
# -*- coding: utf-8 -*-
import chainer
from chainer import functions as F
from chainer import links as L
from chainer import Variable
from chainer import optimizers
from chainer import training
from chainer.training import extensions
import numpy as np
from hyperopt import fmin, tpe, hp
import hyperopt.pyll
from hyperopt.pyll import scope

In [0]:
class MLP(chainer.Chain):

    def __init__(self, n_units1, n_units2, n_units3, n_units4, n_out, layer_num, activate):
        super(MLP, self).__init__(
            l1=L.Linear(None, n_units1),
            l2=L.Linear(None, n_units2),
            l3=L.Linear(None, n_units3),
            l4=L.Linear(None, n_units4),
            lfinal=L.Linear(None, n_out),
        )
        self.layer_num = layer_num
        if activate == 'relu':
            self.act = F.relu
        else:
            self.act = F.sigmoid

    def __call__(self, x):
        h1 = self.act(self.l1(x))
        h2 = self.act(self.l2(h1))
        if self.layer_num == 3:
            return self.lfinal(h2)
        h3 = self.act(self.l3(h2))
        if self.layer_num == 4:
            return self.lfinal(h3)
        h4 = self.act(self.l4(h3))
        if self.layer_num == 5:
            return self.lfinal(h4)
          

In [15]:
def main(params):
    epoch = 40
    gpu = 0
    n_out = 10
    batchsize = 100

    n_units1 = params['n_units1']
    n_units2 = params['n_units2']
    n_units3 = params['n_units3']
    n_units4 = params['n_units4']
    layer_num = params['layer_num']
    activate = params['activate']
    optimizer_name = params['optimizer_name']
    lr = params['lr']

    model = L.Classifier(MLP(n_units1, n_units2, n_units3, n_units4, n_out, layer_num,
                 activate))
    if gpu >= 0:
        chainer.cuda.get_device(gpu).use()
        model.to_gpu()

    # Setup an optimizer
    if optimizer_name == 'Adam':
        optimizer = chainer.optimizers.Adam()
    elif optimizer_name == 'AdaDelta':
        optimizer = chainer.optimizers.AdaDelta()
    else:
        optimizer = chainer.optimizers.MomentumSGD(lr=lr)
    optimizer.setup(model)
    # optimizer.add_hook(chainer.optimizer.WeightDecay(weight_decay))

    # Load the MNIST dataset
    train, test = chainer.datasets.get_mnist()

    train_iter = chainer.iterators.SerialIterator(train, batchsize)
    test_iter = chainer.iterators.SerialIterator(test, batchsize,
                                                 repeat=False, shuffle=False)

    # Set up a trainer
    updater = training.StandardUpdater(train_iter, optimizer, device=gpu)
    trainer = training.Trainer(updater, (epoch, 'epoch'), out="result")

    # Evaluate the model with the test dataset for each epoch
    trainer.extend(extensions.Evaluator(test_iter, model, device=gpu))

    # Write a log of evaluation statistics for each epoch
    trainer.extend(extensions.LogReport())

    # Save two plot images to the result dir
    trainer.extend(
        extensions.PlotReport(['main/loss', 'validation/main/loss'],
                               'epoch', file_name='loss.png'))
    trainer.extend(
        extensions.PlotReport(
            ['main/accuracy', 'validation/main/accuracy'],
             'epoch', file_name='accuracy.png'))

    # Write report
    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

    # Print a progress bar to stdout
    trainer.extend(extensions.ProgressBar())

    # Run the training
    trainer.run()
    valid_data = trainer._extensions['PlotReport'].extension._data
    loss_data = [data for i, data in valid_data['validation/main/loss']]
    best10_loss = sorted(loss_data)[:10]
    return sum(best10_loss)


if __name__ == '__main__':
    # params = {'n_units1': 200,
    #          'n_units2': 200,
    #          'n_units3': 200,
    #          'n_units4': 200,
    #          'layer_num': 3,
    #          'activate': 'relu',
    #          'optimizer_name': 'Adam',
    #          'lr': 0.01,
    #          }
    # main(params)
    space = {'n_units1': scope.int(hp.quniform('n_units1', 100, 300, 50)),
             'n_units2': scope.int(hp.quniform('n_units2', 100, 300, 50)),
             'n_units3': scope.int(hp.quniform('n_units3', 100, 300, 50)),
             'n_units4': scope.int(hp.quniform('n_units4', 100, 300, 50)),
             'layer_num': scope.int(hp.quniform('layer_num', 3, 5, 1)),
             'activate': hp.choice('activate',
                                         ('relu', 'sigmoid')),
             'optimizer_name': hp.choice('optimizer_name',
                                         ('Adam', 'AdaDelta', 'MomentumSGD')),
             'lr': hp.uniform('lr', 0.005, 0.02),
             }
    best = fmin(main, space, algo=tpe.suggest, max_evals=20)
    print("best parameters", best)

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [..................................................]  0.42%
this epoch [########..........................................] 16.67%
       100 iter, 0 epoch / 40 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.83%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 40 epochs
    126.57 iters/sec. Estimated time to finish: 0:03:08.044220.
     total [..................................................]  1.25%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 40 epochs
    126.35 iters/sec. Estimated time to finish: 0:03:07.579052.
     total [..................................................]  1.67%
this epoch [#################################.................] 66.67%
       400 iter, 0 epoch / 40 epochs

     total [#####.............................................] 11.25%
this epoch [#########################.........................] 50.00%
      2700 iter, 4 epoch / 40 epochs
    109.21 iters/sec. Estimated time to finish: 0:03:15.045736.
     total [#####.............................................] 11.67%
this epoch [#################################.................] 66.67%
      2800 iter, 4 epoch / 40 epochs
    109.71 iters/sec. Estimated time to finish: 0:03:13.233160.
     total [######............................................] 12.08%
this epoch [#########################################.........] 83.33%
      2900 iter, 4 epoch / 40 epochs
    110.17 iters/sec. Estimated time to finish: 0:03:11.526450.
5           0.0960216   0.102506              0.971982       0.9668                    27.3482       
     total [######............................................] 12.50%
this epoch [..................................................]  0.00%
      3000 iter, 5 epoch / 

     total [###########.......................................] 22.08%
this epoch [#########################################.........] 83.33%
      5300 iter, 8 epoch / 40 epochs
    108.88 iters/sec. Estimated time to finish: 0:02:51.746247.
9           0.0423559   0.0721967             0.987882       0.9772                    49.7098       
     total [###########.......................................] 22.50%
this epoch [..................................................]  0.00%
      5400 iter, 9 epoch / 40 epochs
    107.47 iters/sec. Estimated time to finish: 0:02:53.078282.
     total [###########.......................................] 22.92%
this epoch [########..........................................] 16.67%
      5500 iter, 9 epoch / 40 epochs
    107.73 iters/sec. Estimated time to finish: 0:02:51.717811.
     total [###########.......................................] 23.33%
this epoch [################..................................] 33.33%
      5600 iter, 9 epoch / 

     total [################..................................] 32.92%
this epoch [########..........................................] 16.67%
      7900 iter, 13 epoch / 40 epochs
    107.67 iters/sec. Estimated time to finish: 0:02:29.524409.
     total [################..................................] 33.33%
this epoch [################..................................] 33.33%
      8000 iter, 13 epoch / 40 epochs
    107.86 iters/sec. Estimated time to finish: 0:02:28.347163.
     total [################..................................] 33.75%
this epoch [#########################.........................] 50.00%
      8100 iter, 13 epoch / 40 epochs
    108.01 iters/sec. Estimated time to finish: 0:02:27.206233.
     total [#################.................................] 34.17%
this epoch [#################################.................] 66.67%
      8200 iter, 13 epoch / 40 epochs
    108.18 iters/sec. Estimated time to finish: 0:02:26.055944.
     total [############

     total [#####################.............................] 43.75%
this epoch [#########################.........................] 50.00%
     10500 iter, 17 epoch / 40 epochs
    107.19 iters/sec. Estimated time to finish: 0:02:05.939338.
     total [######################............................] 44.17%
this epoch [#################################.................] 66.67%
     10600 iter, 17 epoch / 40 epochs
    108.03 iters/sec. Estimated time to finish: 0:02:04.033902.
     total [######################............................] 44.58%
this epoch [#########################################.........] 83.33%
     10700 iter, 17 epoch / 40 epochs
    108.12 iters/sec. Estimated time to finish: 0:02:03.009186.
18          0.00630509  0.0812041             0.998766       0.9777                    100.036       
     total [######################............................] 45.00%
this epoch [..................................................]  0.00%
     10800 iter, 18 epoc

...

23          0.00705331  0.102279              0.998083       0.9807                    167.306       
     total [############################......................] 57.50%
this epoch [..................................................]  0.00%
     13800 iter, 23 epoch / 40 epochs
    83.464 iters/sec. Estimated time to finish: 0:02:02.208323.
     total [############################......................] 57.92%
this epoch [########..........................................] 16.67%
     13900 iter, 23 epoch / 40 epochs
    83.454 iters/sec. Estimated time to finish: 0:02:01.024242.
     total [#############################.....................] 58.33%
this epoch [################..................................] 33.33%
     14000 iter, 23 epoch / 40 epochs
    83.454 iters/sec. Estimated time to finish: 0:01:59.826235.
     total [#############################.....................] 58.75%
this epoch [#########################.........................] 50.00%
     14100 iter, 23 epoc

     total [##################################................] 68.33%
this epoch [################..................................] 33.33%
     16400 iter, 27 epoch / 40 epochs
      83.3 iters/sec. Estimated time to finish: 0:01:31.235986.
     total [##################################................] 68.75%
this epoch [#########################.........................] 50.00%
     16500 iter, 27 epoch / 40 epochs
    83.288 iters/sec. Estimated time to finish: 0:01:30.048584.
     total [##################################................] 69.17%
this epoch [#################################.................] 66.67%
     16600 iter, 27 epoch / 40 epochs
    83.822 iters/sec. Estimated time to finish: 0:01:28.282561.
     total [##################################................] 69.58%
this epoch [#########################################.........] 83.33%
     16700 iter, 27 epoch / 40 epochs
    83.819 iters/sec. Estimated time to finish: 0:01:27.092251.
28          0.00710589  

     total [#######################################...........] 79.17%
this epoch [#################################.................] 66.67%
     19000 iter, 31 epoch / 40 epochs
    83.753 iters/sec. Estimated time to finish: 0:00:59.699624.
     total [#######################################...........] 79.58%
this epoch [#########################################.........] 83.33%
     19100 iter, 31 epoch / 40 epochs
    83.756 iters/sec. Estimated time to finish: 0:00:58.502970.
32          0.00824415  0.127844              0.997699       0.9778                    232.32        
     total [########################################..........] 80.00%
this epoch [..................................................]  0.00%
     19200 iter, 32 epoch / 40 epochs
    83.192 iters/sec. Estimated time to finish: 0:00:57.698035.
     total [########################################..........] 80.42%
this epoch [########..........................................] 16.67%
     19300 iter, 32 epoc

36          0.00703811  0.138933              0.998099       0.9768                    261.292       
     total [#############################################.....] 90.00%
this epoch [..................................................]  0.00%
     21600 iter, 36 epoch / 40 epochs
    82.931 iters/sec. Estimated time to finish: 0:00:28.939822.
     total [#############################################.....] 90.42%
this epoch [########..........................................] 16.67%
     21700 iter, 36 epoch / 40 epochs
     82.92 iters/sec. Estimated time to finish: 0:00:27.737630.
     total [#############################################.....] 90.83%
this epoch [################..................................] 33.33%
     21800 iter, 36 epoch / 40 epochs
    82.923 iters/sec. Estimated time to finish: 0:00:26.530706.
     total [#############################################.....] 91.25%
this epoch [#########################.........................] 50.00%
     21900 iter, 36 epoc

best parameters {'activate': 0, 'layer_num': 3.0, 'lr': 0.015151066751728622, 'n_units1': 200.0, 'n_units2': 250.0, 'n_units3': 200.0, 'n_units4': 150.0, 'optimizer_name': 2}
